In [1]:
#Imports
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.functional as F
import pandas as pd
import os
from torchvision.io import read_image
import torchvision.models as models
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
from torchvision.transforms import transforms
from typing import Iterator
import matplotlib.pyplot as plt
from PIL import Image
import time
import copy
import random

In [8]:
#Constants
img_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Images\Food Images"
data_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Ingredients and Recipe Dataset with Image Name Mapping.csv"
batch_size = 128
learning_rate = 0.001
momentum = 0.9
img_size = 169
output_size = 13470

In [3]:
class Sampler(RandomSampler):
    #removes bad values
    
    def __init__(self, data_source, forbidden = []) -> None:
        super().__init__(data_source)
        self.data_source = data_source
        self.forbidden = forbidden
        self.refill()

    def remove(self, new_forbidden):
        # Remove numbers from the available indices
        for num in new_forbidden:
            if not (num in self.forbidden):
                self.forbidden.append(num)
        self._remove(new_forbidden)

    def _remove(self, to_remove):
        # Remove numbers just for this epoch
        for num in to_remove:
            if num in self.idx:
                self.idx.remove(num)

        self._num_samples = len(self.idx)

    def refill(self):
        # Refill the indices after iterating through the entire DataLoader
        self.idx = list(range(len(self.data_source)))
        self._remove(self.forbidden)

    def __iter__(self) -> Iterator[int]:
        for _ in range(self.num_samples // 32):
            batch = random.sample(self.idx, 32)
            self._remove(batch)
            yield from batch
        yield from random.sample(self.idx, self.num_samples % 32)
        self.refill()

In [4]:
# constructing the dataloader
class foodImageDataset(Dataset):
    def __init__(self, img_dir, data_dir, transformation = None):
        self.img_dir = img_dir
        self.ingr = pd.read_csv(data_dir, header=1,usecols=[5])
        self.transform = transformation
        self.labels = pd.read_csv(data_dir, header=1,usecols=[0,4])

    #Later augment the dataset to give more images per class to increase accuracy

    def __len__(self):
       return len(self.labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels.iloc[idx,1] + ".jpeg")
        try:
            image = read_image(img_path)
        except:
            f"file not found {img_path}"
            return None
        label = self.labels.iloc[idx, 0]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __getingrs__(self, index):
        return self.ingr.iloc[index,0]
    

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((169,169)),
    transforms.ToTensor(),
])

food = foodImageDataset(img_dir, data_dir, transformation=transform)
food_dl = DataLoader(food, batch_size, shuffle=True)

# for i in range(len(food)):
#     sample = food[i]
#     if food[i] is  None:
#         print(i)
#         # print(i,sample['image'].shape, sample["label"], sample["ingr"])


In [10]:
device = torch.cuda.set_device(0)
model_ft = models.vgg16(pretrained=True)

classifier = model_ft.classifier
num_ftrs = classifier[0].in_features
model_ft.fc = nn.Linear(num_ftrs, output_size)
criterion = nn.CrossEntropyLoss()

model_ft = model_ft.to(device)
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


def train_model(trainloader,model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
            # Iterate over data.
            for inputs, labels in trainloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
            if phase == 'train':
                scheduler.step()
            # deep copy the model
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(food_dl, model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


IndexError: Target 11776 is out of bounds.